In [1]:
import import_ipynb
import BPVal as bp
import random

In [2]:
class Neuron:
    def __init__(self, nin, act = "relu"):
        self.weights = [bp.BPVal(random.uniform(0, 1)) for _ in range(nin)]
        self.bias = bp.BPVal(0)
        self.act = act

    def __call__(self, ins):
        assert(len(ins) == len(self.weights))
        out = sum((w*x for w,x in zip(ins, self.weights)), self.bias)
        if self.act.lower() == "relu":
            out = out.relu()
        elif self.act.lower() == "sigmoid":
            out = out.sigmoid()
        return out
        
    def parameters(self):
        return self.weights + [self.bias]

In [3]:
class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout-1)] + [Neuron(nin, act="None")]

    def __call__(self, ins):
        outs = [n(ins) for n in self.neurons]
        return outs if len(outs) > 1 else outs[0]

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]
        

In [4]:
class MLP:
    def __init__(self, nin, layer_sizes):
        tmp = [nin] + layer_sizes
        self.layers = [Layer(tmp[i], tmp[i+1]) for i in range(len(layer_sizes))]

    def __call__(self, ins):
        for layer in self.layers:
            ins = layer(ins)
        return ins

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0.0